In [1]:
# 카카오 api 키를 활용해서 주소 -> 좌표, 좌표-> 주소 불러오기
import pandas as pd
import requests 
import sys
import json
import datetime 

In [2]:
KAKAO_API_KEY = "" #카카오 restapi 키 등록
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

In [3]:
# cctv data 불러오기, 주소변경필요
cctv = pd.read_csv("C:/Users/gherwt/KDT30/p1/cctv.csv",  encoding = "cp949")
cctv = cctv.rename(columns={'위도': 'lat'})
cctv = cctv.rename(columns={'경도': 'lng'})
cctv.head(2)

,자치구,안심 주소,lat,lng,CCTV 수량,수정 일시,dong
0,성동구,B010_(고정1)7_뚝섬역 7번출구 주변,37.5474,127.0465,1,2022-12-01,NaN
1,성동구,B012_(고정1)3_뚝섬역 2번출구 주변,37.5476,127.0468,1,2022-12-01,NaN


In [4]:
# url 로 데이터를 보내서 가져오기
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(KAKAO_API_KEY)}
    resp = requests.get(url, headers=headers)
    #print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text

def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    print(url)
    try:
        # print('try')
        json_req = json_request(url=url) # 카카오가 해당 위, 경도에 대한 정보를 전달
        # dic 운영 코드
        json_data = json.loads(json_req) 
        json_doc = json_data.get('documents')[0]
        json_name = json_doc.get('address_name')

    except:
        # print('nan'), nan 값이면 nan 을 넣어줘서 반환한다.
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name


def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address # 위경도 전달, 주소 저장


In [5]:
# 0행부터 마지막행까지 역지오코딩을 실행, 결과를 cctv 의 dong 열에 추가해준다.
for i in range(len(cctv['lng'])):
    lng = cctv['lng'][i]
    lat = cctv['lat'][i]
    address = reverse_geocode(lng, lat)
    cctv['dong'][i] = address

https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0465&y=37.5474
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0468&y=37.5476
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0468&y=37.5476
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0548&y=37.5448
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0548&y=37.5448


C:\Users\gherwt\AppData\Local\Temp\ipykernel_36048\2087654168.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cctv['dong'][i] = address
C:\Users\gherwt\AppData\Local\Temp\ipykernel_36048\2087654168.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'NaN' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  cctv['dong'][i] = address


https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0558&y=37.5445
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0558&y=37.5445
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0568&y=37.5442
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0568&y=37.5442
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0568&y=37.5442
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0549&y=37.5451
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0549&y=37.5451
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0547&y=37.5608
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0547&y=37.5608
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0547&y=37.5608
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0542&y=37.5627
https://dapi.kakao.com/v2/local/geo/coord2regioncode.json?x=127.0542&y=37.5627
https://dapi.kakao.com/v2/local/geo/coord2regioncode

In [6]:
# 자료 완료시 성동구에 해당하지 않는 법정동이 존재 이를 제거해준다.
cctv = cctv[cctv['dong'] != '신당동']
cctv = cctv.rename(columns={'CCTV 수량': 'cctv_cnt'})

In [7]:
# cctv 를 csv 로 저장, 주소변경필요
cctv.to_csv('C:/Users/gherwt/KDT30/p1/cctv_data.csv',index=False)

In [8]:
# 동 기준으로 동마다 cctv 가 몇 개 있는지를 확인하기 위해 groupby, count 실행
cctv_cnt = cctv.groupby('dong').count()
cctv_cnt = cctv_cnt['cctv_cnt']
cctv_cnt

dong
NaN    4036
Name: cctv_cnt, dtype: int64